In [14]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np

def process_windprofiler(filepath, filename):
    ### GET HTML INFO
    # Read the HTML content from the file
    with open(filepath, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the <pre> tag and extract its text
    pre_tag = soup.find('pre')
    if pre_tag:
        text_content = pre_tag.get_text()
    else:
        text_content = "No <pre> tag found in the HTML content."

    # Split the text content into lines
    lines = text_content.splitlines()
    
    ### GET UPPER PART STATIC INFO
    # Assuming 'lines' is a list of the lines containing the relevant metadata
    metadata = {}
    station_line = None
    date_line = None

    # Find the index of lines which contain 'Station:' and 'Date:' respectively
    for i, line in enumerate(lines):
        if 'Station:' in line:
            station_line = i
        if 'Date:' in line:
            date_line = i

    # Extract information based on identified line
    if station_line is not None:
        station_match = re.search(r'Station:\s+(.*?)\s{2,}', lines[station_line])
        if station_match:
            if "Sha Lo Wan - Chek Lap Kok Airpt." in station_match.group(1):
                station_name = "SLW"

            if "Siu Ho Wan"  in station_match.group(1):
                station_name = "SHW"

            metadata['Station'] = station_name

    if date_line is not None:
        # Use direct string slicing based on the position
        metadata['Date'] = lines[date_line][20:28].strip()

    # Convert metadata to string if needed
    metadata_str = ""
    if 'Date' in metadata:
        metadata_str += "Date: {}\n".format(metadata['Date'])
    if 'Station' in metadata:
        metadata_str += "Station: {}".format(metadata['Station'])
    
    
    ### Process the lower part for CSV data
    # Find the start of the data section
    data_start_idx = next(i for i, line in enumerate(lines) if "(m/s)" in line) + 1
    data_lines = lines[data_start_idx:]
    
    # CLEAN LINES
    def clean_split(line):
        # Normalize all sequences of spaces or tabs to exactly three spaces
        normalized_line = re.sub(r'[\t\s]+', '   ', line.strip())
        # Split the line based on three spaces
        columns = normalized_line.split('   ')
        return columns

    # Assuming 'data_lines' is a list of the lines containing the relevant data
    data = [clean_split(line) for line in data_lines if line.strip()]

    # Convert to a DataFrame
    df = pd.DataFrame(data)


    if "SLW" in station_name:
        headers = ["Code", "height_(m agl)", 
                   "ws_(m/s)", 
                   "wd_(deg)", 
                   "u_(m/s)", 
                   "v_(m/s)", 
                   "w_(m/s)", 
                   "cns_N", 
                   "cns_W", 
                   "cns_V", 
                   "snr_N", 
                   "snr_W", 
                   "snr_V"]
    else:
        headers = ["Code", 
                   "height_(m agl)", 
                   "ws_(m/s)", 
                   "wd_(deg)", 
                   "u_(m/s)", 
                   "v_(m/s)", 
                   "w_(m/s)", 
                   "cns_NE", 
                   "cns_SE", 
                   "cns_V", 
                   "snr_NE", 
                   "snr_SE", 
                   "snr_V"]
    df.columns = headers

    ### FILL THE MISSING CODE
    def fill_zeros_with_first_value(column):
        # Replace ' 0' strings with None
        column_replaced = column.apply(lambda x: None if x == ' 0' else x)
        # Set non 4-digit cells to None
        column_replaced = column_replaced.apply(lambda x: x if x is not None and re.fullmatch(r'\d{4}', x) else None)
        # Forward-fill None values with the first non-None value encountered
        column_filled = column_replaced.fillna(method='ffill')
        return column_filled

    new_df = df.copy()

    # Apply the function to the 'Code' column
    new_df['Code'] = fill_zeros_with_first_value(new_df['Code'])

    # Specify the columns you're interested in
    columns_of_interest = ["u_(m/s)", "v_(m/s)", "w_(m/s)"]
    # Drop the rows where all elements in the columns of interest are None
    new_df = new_df.dropna(subset=columns_of_interest, how='all')
    
    ### PIVOT TIME
    # Prepare the station name by replacing spaces with underscores.
    station_name = metadata['Station'].replace(" ", "_")

    # Convert date from 'mmddyy' to a 'yyyy-mm-dd' format.
    date_str = metadata['Date']
    date_dt = pd.to_datetime(date_str, format='%m/%d/%y').strftime('%Y-%m-%d')

    # Function to convert the 'Code' to a datetime format.
    def convert_to_datetime(code, date_str):
        # Add leading zeros to the 'Code' if it's not four digits long.
        code_str = str(code).zfill(4)
        # Combine the 'Code' with 'date_str' (formatted as 'yyyy-mm-dd').
        datetime_str = f"{date_str} {code_str[:2]}:{code_str[2:]}:00"
        return pd.to_datetime(datetime_str)

    # Apply the function to create a new 'datetime' column.
    new_df['datetime'] = new_df['Code'].apply(lambda code: convert_to_datetime(code, date_dt))

    # Create a multi-index by combining 'datetime' with 'height_(m agl)'.
    new_df.set_index(['datetime', 'height_(m agl)'], inplace=True)
    
    # 重置索引以便 'height_(m agl)' 成为一个列
    new_df_reset = new_df.reset_index()
    new_df_reset = new_df_reset.drop(columns=['Code'])
    # 现在 'height_(m agl)' 应该是列之一，可以进行数据透视
    df_pivot = new_df_reset.pivot(index='datetime', columns='height_(m agl)')
    # 你可能需要再次重命名列名以便包含站点名称和测量类型
    df_pivot.columns = [f"{station_name}_{col[0]}_{col[1]}" for col in df_pivot.columns]
    df_pivot = df_pivot.reset_index()
    
    ### OUTPUT 
    # Save the pivot DataFrame to a CSV file
    output_path = "C:\\Users\\User\\Desktop\\Final Year Project\\Code\\Processed_HKO\\processing_windprofiler"
    output_filename = f"{metadata['Station']}_windprofiler_{filename}_{date_dt.replace('-', '')}.csv"
    df_pivot.to_csv(f"{output_path}\\{output_filename}", index=False)

In [17]:
import os
from tqdm import tqdm  # Make sure to import the tqdm function

directories = [
    "C:\\Users\\User\\Desktop\\Final Year Project\\HKO Raw Data\\windprofiler\\2018",
    "C:\\Users\\User\\Desktop\\Final Year Project\\HKO Raw Data\\windprofiler\\2019",
    "C:\\Users\\User\\Desktop\\Final Year Project\\HKO Raw Data\\windprofiler\\2020",
    "C:\\Users\\User\\Desktop\\Final Year Project\\HKO Raw Data\\windprofiler\\2021",
    "C:\\Users\\User\\Desktop\\Final Year Project\\HKO Raw Data\\windprofiler\\2022",
    "C:\\Users\\User\\Desktop\\Final Year Project\\HKO Raw Data\\windprofiler\\2023"
]

def process_all_windprofiler_files(directory):
    # Retrieve a list of text files in the given directory
    txt_files = [f for f in os.listdir(directory) if f.endswith('.txt')]

    # Loop through the files with tqdm for a progress bar
    for filename in tqdm(txt_files, desc=f"Processing files in {directory}"):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        new_filename = filename.replace(".txt", "")

        # Process the file
        try:
            process_windprofiler(file_path, new_filename)
        except Exception as e:
            print(f"Failed to process file: {filename}. Error: {e}")
            
# Loop over the directories and process all files in each directory
for directory in directories:
    print(f"Processing directory: {directory}")
    process_all_windprofiler_files(directory)

Processing directory: C:\Users\User\Desktop\Final Year Project\HKO Raw Data\windprofiler\2018


Processing files in C:\Users\User\Desktop\Final Year Project\HKO Raw Data\windprofiler\2018: 100%|██████████| 728/728 [03:25<00:00,  3.55it/s]


Processing directory: C:\Users\User\Desktop\Final Year Project\HKO Raw Data\windprofiler\2019


Processing files in C:\Users\User\Desktop\Final Year Project\HKO Raw Data\windprofiler\2019: 100%|██████████| 716/716 [03:20<00:00,  3.58it/s]


Processing directory: C:\Users\User\Desktop\Final Year Project\HKO Raw Data\windprofiler\2020


...

Processing files in C:\Users\User\Desktop\Final Year Project\HKO Raw Data\windprofiler\2023:  43%|████▎     | 260/608 [01:17<01:42,  3.39it/s]

Failed to process file: clk30918.txt. Error: Length mismatch: Expected axis has 0 elements, new values have 13 elements


Processing files in C:\Users\User\Desktop\Final Year Project\HKO Raw Data\windprofiler\2023:  98%|█████████▊| 597/608 [02:48<00:02,  3.76it/s]

Failed to process file: shw31020.txt. Error: Index contains duplicate entries, cannot reshape


Processing files in C:\Users\User\Desktop\Final Year Project\HKO Raw Data\windprofiler\2023: 100%|██████████| 608/608 [02:51<00:00,  3.54it/s]


In [18]:
import os
import pandas as pd

# Define input and output directories
input_path = "C:\\Users\\User\\Desktop\\Final Year Project\\Code\\Processed_HKO\\processing_windprofiler"
output_path = "C:\\Users\\User\\Desktop\\Final Year Project\\Code\\Processed_HKO\\processed_wp"

# Ensure the output directory exists, create if it doesn't
os.makedirs(output_path, exist_ok=True)

# List of prefixes
prefixes = ["SHW_", "SLW_"]

for file_prefix in prefixes:
    # Initialize an empty list to store DataFrames for the current prefix
    df_list = []

    # Loop through all the files in the input directory
    for filename in os.listdir(input_path):
        # Check if the file starts with the current prefix
        if filename.startswith(file_prefix):
            # Construct the full file path
            file_path = os.path.join(input_path, filename)
            # Read the CSV file and append it to the list
            df_list.append(pd.read_csv(file_path))

    # Concatenate all DataFrames in the list into a single DataFrame
    concatenated_df = pd.concat(df_list, ignore_index=True)

    # Construct the output file path
    concatenated_filename = f"{file_prefix}windprofiler_concatenated.csv"
    output_file_path = os.path.join(output_path, concatenated_filename)

    # Save the concatenated DataFrame to a single CSV file in the output directory
    concatenated_df.to_csv(output_file_path, index=False)

    print(f"All files starting with {file_prefix} have been concatenated into {output_file_path}")

All files starting with SHW_ have been concatenated into C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\processed_wp\SHW_windprofiler_concatenated.csv
All files starting with SLW_ have been concatenated into C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\processed_wp\SLW_windprofiler_concatenated.csv


### CONCAT BOTH SHW SLW

In [3]:
%%time
import os
import pandas as pd

# Define the directory containing the files
directory = "C:\\Users\\User\\Desktop\\Final Year Project\\Code\\Processed_HKO\\processed_wp"

# Define filenames based on the prefixes used earlier
shw_filename = os.path.join(directory, "SHW_windprofiler_concatenated.csv")
slw_filename = os.path.join(directory, "SLW_windprofiler_concatenated.csv")

# Read the files into DataFrames
shw_df = pd.read_csv(shw_filename)
slw_df = pd.read_csv(slw_filename)

# Convert 'datetime' columns to pandas datetime objects and round to the nearest minute
shw_df['datetime'] = pd.to_datetime(shw_df['datetime']).dt.round('min')
slw_df['datetime'] = pd.to_datetime(slw_df['datetime']).dt.round('min')

# Sort the DataFrames by 'datetime' in ascending order
shw_df = shw_df.sort_values('datetime', ascending=True)
slw_df = slw_df.sort_values('datetime', ascending=True)

# Perform the asof merge with 'nearest' direction and within a 3-minute tolerance
joined_df = pd.merge_asof(shw_df, slw_df, on='datetime', direction='nearest', tolerance=pd.Timedelta('3min'))

# Save the joined DataFrame to a new CSV file
output_filename = os.path.join(directory, "Joined_SHW_SLW_windprofiler.csv")
joined_df.to_csv(output_filename, index=False)

print(f"The files have been successfully joined and saved to {output_filename}")

The files have been successfully joined and saved to C:\Users\User\Desktop\Final Year Project\Code\Processed_HKO\processed_wp\Joined_SHW_SLW_windprofiler.csv
CPU times: total: 2min 8s
Wall time: 2min 9s
